<a href="https://colab.research.google.com/github/ahakobia/Group4_NFLX_MIDTERM/blob/main/Adehs_Pyspark_US_State_Energy_NEW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Activate Spark in our Colab notebook.
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example: 'spark-3.2.2'
spark_version = 'spark-3.2.2'
# spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.2.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.2.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3.2"

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Get:12 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [2,992 kB]
Hit:13 http://ppa.la

In [2]:
# Install pytest and pytest-sugar to make our output look nice.
!pip install -q pytest pytest-sugar

In [3]:
# Create and navigate to the tdd directory.
from pathlib import Path
if Path.cwd().name != 'tests':
    %mkdir tests
    %cd tests
# Show the current working directory.  
%pwd

/content/tests


'/content/tests'

In [4]:
# Initialize the __init__.py file. 
# This file will be stored in our pwd (/content/tests)
%%file __init__.py
pass

Writing __init__.py


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [17]:
import findspark
findspark.init()

# Import other dependencies. 
from pyspark import SparkFiles
from pyspark.sql import SparkSession

import time

spark = SparkSession \
    .builder \
    .appName("energy file read in") \
    .getOrCreate()
energy = spark.read.csv('../drive/MyDrive/organised_Gen.csv', header = True, inferSchema = True)
energy = energy.withColumnRenamed('_c0', 'id')
energy = energy.withColumnRenamed('YEAR', 'year')
energy = energy.withColumnRenamed('MONTH', 'month')
energy = energy.withColumnRenamed('TYPE OF PRODUCER', 'producer')
energy = energy.withColumnRenamed('ENERGY SOURCE', 'source')
energy = energy.withColumnRenamed('GENERATION (Megawatthours)', 'generated')
energy.write.parquet('parquet_energy',mode='overwrite')
p_energy = spark.read.parquet('parquet_energy')
p_energy = p_energy.where("producer == 'Total Electric Power Industry' AND source == 'Total' AND state != 'US-TOTAL AND year != 2022'")
p_energy.createOrReplaceTempView('p_energy_data')

start_time = time.time()
  
q_df = spark.sql("""
      SELECT year, state, SUM(generated) as Total_generated
            FROM p_energy_data 
            Group By year, state
            Order By year DESC, Total_generated DESC
                """)
print("--- %s seconds ---" % (time.time() - start_time))

q_df.show()

--- 0.01768660545349121 seconds ---
+----+--------+---------------+
|year|   state|Total_generated|
+----+--------+---------------+
|2022|US-TOTAL|  1.680536449E9|
|2022|      TX|   2.02632121E8|
|2022|      PA|    9.8886241E7|
|2022|      FL|    9.8030052E7|
|2022|      IL|     7.797494E7|
|2022|      CA|    7.4409695E7|
|2022|      AL|    5.8449245E7|
|2022|      OH|    5.4393347E7|
|2022|      NC|    5.2738148E7|
|2022|      GA|    5.0216005E7|
|2022|      WA|    4.9491911E7|
|2022|      NY|    4.9361595E7|
|2022|      MI|    4.4213685E7|
|2022|      LA|     4.158867E7|
|2022|      SC|    4.0976847E7|
|2022|      IN|    4.0925857E7|
|2022|      AZ|    3.7998697E7|
|2022|      VA|    3.4972164E7|
|2022|      TN|    3.3679753E7|
|2022|      OK|    3.3396004E7|
+----+--------+---------------+
only showing top 20 rows



In [18]:
# Create a bank_data.py file and write the function to it. 
# This file will be stored in our pwd (/content/tests).
%%file total_energy.py

# Import findspark() and initialize. 
import findspark
findspark.init()

# Import other dependencies. 
from pyspark import SparkFiles
from pyspark.sql import SparkSession

# Create the import_data function. 
def import_data():
    spark = SparkSession \
    .builder \
    .appName("energy file read in") \
    .getOrCreate()
    energy = spark.read.csv('../drive/MyDrive/organised_Gen.csv', header = True, inferSchema = True)
    energy = energy.withColumnRenamed('_c0', 'id')
    energy = energy.withColumnRenamed('YEAR', 'year')
    energy = energy.withColumnRenamed('MONTH', 'month')
    energy = energy.withColumnRenamed('TYPE OF PRODUCER', 'producer')
    energy = energy.withColumnRenamed('ENERGY SOURCE', 'source')
    energy = energy.withColumnRenamed('GENERATION (Megawatthours)', 'generated')
    energy.write.parquet('parquet_energy',mode='overwrite')
    return energy

def transform_data():
    spark = SparkSession \
    .builder \
    .appName("energy file read in") \
    .getOrCreate()
    p_energy = spark.read.parquet('parquet_energy')
    p_energy = p_energy.where("producer == 'Total Electric Power Industry' AND source == 'Total' AND state != 'US-TOTAL AND year != 2022'")
    p_energy.createOrReplaceTempView('p_energy_data')
    return p_energy

def query_data():
    spark = SparkSession \
    .builder \
    .appName("energy file read in") \
    .getOrCreate()
    q_df = spark.sql("""
      SELECT year, state, SUM(generated) as Total_generated
            FROM p_energy_data 
            Group By year, state
            Order By year DESC, Total_generated DESC
                """)
    return q_df


Overwriting total_energy.py


In [23]:
# Create a test_bank_data.py file and write the test functions to it. 
# This file will be stored in our pwd (/content/tests).
%%file test_total_states_energy.py

from total_energy import (import_data, transform_data, query_data)

# Write the tests. 
def test_row_count_before_transform():
  df = import_data()
  assert df.count() == 496774

def test_column_count_before_transform():
  df = import_data()
  assert len(df.columns) == 7

def test_row_count_after_transform():
    df = transform_data()
    assert df.count() != 496774

def test_column_count_after_transform():
    df = transform_data()
    assert len(df.columns) == 7

def test_row_count_query():
    df = query_data()
    assert df.count() == 1144

def test_column_count_query():
    df = query_data()
    assert len(df.columns) == 3


Overwriting test_total_states_energy.py


In [24]:
# Run the test_import_data.py file with pytest. 
!python -m pytest test_total_states_energy.py

Test session starts (platform: linux, Python 3.7.14, pytest 3.6.4, pytest-sugar 0.9.5)
rootdir: /content/tests, inifile:
plugins: typeguard-2.7.1, sugar-0.9.5

 test_total_states_energy.py ✓✓✓✓✓✓                              100% ██████████

Results (34.13s):
       6 passed
